In [1]:
import pandas as pd
import nibabel as nib
import numpy as np

# Define Ontology

In order to create the cortical maps, we have manually matched the ontologies between the modified broadmann atlas in two stages of development - 8 weeks post conception and adult brain (40 years old) - and the 3D MRI Brodmann atlas by Pijnenburg et al (2021). The matching can be found on supplement X. Only cortical areas were included in the plot for consistency of atlas usage. 

In [2]:
# Correspondence between brain areas
brodmann = {
    # Cortical
    'DFC' : ['BA46', 'BA9$'],
    'ITC' : ['BA20'],
    'VFC' : ['BA45'],
    'A1C' : ['BA41_42_52'],
    'V1C' : ['BA17'],
    'OFC' : ['BA10', 'BA11', 'BA47'],
    'STC' : ['BA22'],
    'IPC' : ['BA39', 'BA40'],
    'M1C' : ['BA4$'],
    'S1C' : ['BA1_3'],
    'MFC' : ['BA32', 'BA33'],
    'STC' : ['BA22'],
    'M1C-S1C' : ['BA1_3', 'BA4$'],
    'ITC' : ['BA20'],
    'Ocx' : ['BA17', 'BA18', 'BA19'],
    'PCx' : ['BA2$', 'BA7$', 'BA39', 'BA40'],
    # Non-cortical
    'HIP' : [],
    'STR' : [],
    'MD' : [],
    'AMY' : [],
    'CBC' : [],
    'DTH' : [],
    # Transient
    'LGE' : [],
    'MGE' : [],
    'CGE' : [],
    'URL' : []
}


# Open and format LUT + Atlas files


In [3]:
nii = nib.load('Pijnenburg_et_al_2021_NeuroImage/AtlasCollection/Brodmann/Brodmann_ICBM152.nii.gz')
img = nii.get_fdata()
lut = pd.read_csv('Pijnenburg_et_al_2021_NeuroImage/AtlasCollection/Brodmann/BrodmannColorLUT.txt', comment='#', sep = '\s+', index_col=0)
lut.columns = ['name','id', 'r', 'g', 'b']
img.shape

(256, 256, 256, 1)

# Project values onto cortical surface

In [8]:
for prefix in ['pre0', 'pre1', 'post0', 'post1']:
    expression_file = f'expression_files/{prefix}.tsv'
    data = pd.read_csv(expression_file, sep = '\t')

    # Match Ontologies
    data['brodmann_id'] = data.structure_acronym.apply(
        lambda x:
          np.array(
              [lut.index[lut.name.str.contains(brod)] for brod in brodmann[x]]
            ).flatten()
    )
    # Create new image
    img_new = img.copy()
    # Input expression values in new image
    for ii, row in data.iterrows():
        for value in row.brodmann_id:
            img_new[img_new == value] = row.expression
    # Zero non parcellated voxels
    img_new[img_new > 2] = 0
    # Save inputed image
    new_nii = nib.Nifti1Image(img_new, nii.affine, nii.header)
    nib.save(new_nii, f'output/nii/{prefix}.nii.gz')

In [9]:
data

,structure_acronym,structure_name,structure_id,expression,brodmann_id
0,DFC,dorsolateral prefrontal cortex,10173,1.075686,"[1038, 2038, 1008, 2008]"
1,ITC,"inferolateral temporal cortex (area TEv, area 20)",10252,1.070808,"[1016, 2016]"
2,VFC,ventrolateral prefrontal cortex,10185,1.458505,"[1037, 2037]"
3,MD,mediodorsal nucleus of thalamus,10398,0.993099,[]
4,AMY,amygdaloid complex,10361,0.882942,[]
5,A1C,primary auditory cortex (core),10236,0.964009,"[1034, 2034]"
6,CBC,cerebellar cortex,10657,1.110854,[]
7,V1C,"primary visual cortex (striate cortex, area V1...",10269,1.195907,"[1013, 2013]"
8,OFC,orbital frontal cortex,10194,1.423177,"[1009, 2009, 1010, 2010, 1039, 2039]"
9,STC,posterior (caudal) superior temporal cortex (a...,10243,1.408881,"[1018, 2018]"
